This dataset contains negative sentences and positive sentences which are classified by some context
    we have to predict the or to classify the whether it is positive sentence or nagetive sentence. a kind of text classification
    

In [36]:
import numpy as np
import pandas as pd
import re
import pickle
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rajkumar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
from sklearn.datasets import load_files
reviews = load_files('txt_sentoken/')
X,y = reviews.data,reviews.target

In [8]:
type(reviews)


sklearn.utils.Bunch

In [9]:

# to process fast pickling the dataset
with open('X.pickle','wb') as f:
    pickle.dump(X,f)
    
with open('y.pickle','wb') as f:
    pickle.dump(y,f)

In [14]:
# Unpickling dataset
X_in = open('X.pickle','rb')
y_in = open('y.pickle','rb')
X = pickle.load(X_in)
y = pickle.load(y_in)
len(X)

2000

In [15]:
# Creating the corpus
corpus = []
for i in range(0, len(X)):
    review = re.sub(r'\W', ' ', str(X[i]))
    review = review.lower()
    review = re.sub(r'^br$', ' ', review)
    review = re.sub(r'\s+br\s+',' ',review)
    review = re.sub(r'\s+[a-z]\s+', ' ',review)
    review = re.sub(r'^b\s+', '', review)
    review = re.sub(r'\s+', ' ', review)
    corpus.append(review)   

In [21]:
# Creating the BOW model
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = len(X), min_df = 3, max_df = 0.6, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(corpus).toarray()

In [22]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [2, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [25]:

# Creating the Tf-Idf Model
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
X = transformer.fit_transform(X).toarray()
X


# Creating the Tf-Idf model directly
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 2000, min_df = 3, max_df = 0.6, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(corpus).toarray()
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.06887219, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.12007883, 0.        , 0.06321361, ..., 0.        , 0.        ,
        0.        ]])

In [26]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
text_train, text_test, sent_train, sent_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [28]:
# Training the classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(text_train,sent_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [35]:
# Testing model performance
sent_pred = classifier.predict(text_test)

print(sent_pred)
print("\n")
print(sent_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(sent_test, sent_pred)

[1 0 1 1 0 0 1 1 1 1 0 1 0 0 1 1 0 0 1 1 0 1 1 1 0 1 1 0 1 0 1 0 1 0 1 1 0
 0 0 1 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 0 0 0 0 1 0 0 1 0 1
 0 0 0 1 0 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 0 1
 0 1 1 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 1 1 1 0 1 1 0 0 0
 1 1 0 1 0 1 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 0 0 1 1 1 0 1 0
 0 1 1 1 1 1 0 1 0 0 1 0 0 1 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 0 0 1 1 1
 0 0 1 1 0 0 1 0 0 1 1 1 0 0 0 1 0 1 0 0 1 0 1 0 1 0 1 1 1 1 1 1 0 1 0 1 0
 1 1 0 1 0 0 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 0
 0 0 1 1 1 0 0 0 1 0 1 0 1 0 0 1 0 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 0 1
 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 0 0 0 1 1 0 0 1 1 0 1 0 0 1 0 0 1 0 1 0 1 0
 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0 1 0 0 1 1 1 1 0 1 0 0 1 0]


[0 0 1 1 0 0 1 1 1 1 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 1 1 1 0 1 0 1 0 0
 0 0 0 1 1 0 1 1 1 0 0 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 0 0 0 0 1 0 0 1 0 1
 1 0 0 1 0 0 1 1 0 0 1 0 1 1 0 0 0 1

In [30]:
cm

array([[168,  40],
       [ 21, 171]], dtype=int64)

In [31]:

# Saving our classifier
with open('classifier.pickle','wb') as f:
    pickle.dump(classifier,f)
    
# Saving the Tf-Idf model
with open('tfidfmodel.pickle','wb') as f:
    pickle.dump(vectorizer,f)